# Plotting routine for the March 23 2023 Event - Wind

In [ ]:
test = 'test'

In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocessing
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


#### setting the times for py3DCORE

In [13]:
t_launch = datetime.datetime(2023, 3, 20, 7, tzinfo=datetime.timezone.utc) # 

t_s = datetime.datetime(2023, 3, 23, 17, tzinfo=datetime.timezone.utc) 
t_e = t_s + datetime.timedelta(hours=22)

t_fit = [
        datetime.datetime(2023, 3, 23, 18, tzinfo=datetime.timezone.utc),
        datetime.datetime(2023, 3, 23, 19, tzinfo=datetime.timezone.utc),
        #datetime.datetime(2023, 3, 23, 20, tzinfo=datetime.timezone.utc),
        datetime.datetime(2023, 3, 23, 21, tzinfo=datetime.timezone.utc)
        #datetime.datetime(2023, 3, 24, 0, tzinfo=datetime.timezone.utc)
]

In [14]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['03/23/2023, 18:00:00', '03/23/2023, 19:00:00', '03/23/2023, 21:00:00']


#### loading the pickle files

In [15]:
path = 'output/wind23032023_heeq_512_forecast_1/'

filepath = fp.loadpickle(path, number=-1)

In [16]:
filepath[:-7]

'output/wind23032023_heeq_512_forecast_1/07'

#### extracting parameters

In [6]:
res, allres, ind, meanparams = fp.get_params(filepath, give_mineps=True)

INFO:py3dcore.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore.fluxplot: --lon:  3.02
INFO:py3dcore.fluxplot: --lat:  -24.68
INFO:py3dcore.fluxplot: --inc:  107.24
INFO:py3dcore.fluxplot: --diameter 1 AU:  0.14
INFO:py3dcore.fluxplot: --aspect ratio:  4.22
INFO:py3dcore.fluxplot: --launch radius:  22.05
INFO:py3dcore.fluxplot: --launch speed:  699.26
INFO:py3dcore.fluxplot: --t factor:  114.64
INFO:py3dcore.fluxplot: --expansion rate:  1.14
INFO:py3dcore.fluxplot: --magnetic field decay rate:  1.64
INFO:py3dcore.fluxplot: --magnetic field 1 AU:  17.36
INFO:py3dcore.fluxplot: --drag coefficient:  1.24
INFO:py3dcore.fluxplot: --background sw speed:  403.35


In [7]:
out = fp.get_ensemble_stats(filepath)

In [8]:
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,0.375531,-29.880943,95.744400,0.137046,3.412383,19.701057,925.514343,78.116302,16.623165,1.176521,423.907684
std,11.546754,7.882624,12.828524,0.046025,0.971612,5.423337,274.424683,119.450523,2.301805,0.459195,60.991405
min,-38.982460,-50.919437,60.349968,0.051715,1.015684,10.004216,500.328705,-246.172043,8.317690,0.205082,217.705780
25%,-7.466773,-35.068463,87.587103,0.104659,2.737063,15.298275,695.377609,1.043524,15.147584,0.808989,390.515244
50%,0.550856,-30.143749,96.188087,0.135337,3.524668,19.632362,897.027710,95.861176,16.655668,1.172967,422.516479
75%,8.732969,-24.521104,104.481525,0.161737,4.194755,24.147848,1139.022522,175.746277,18.020205,1.547842,461.705528
max,30.801569,-7.651307,138.279663,0.346230,4.992827,29.945738,1498.070923,249.502945,23.721426,1.992343,626.725281


In [9]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [10]:
names = ['lon: ', 'lat: ', 'inc: ', 'diameter 1 AU: ', 'aspect ratio: ', 'launch radius: ', 'launch speed: ', 
         't factor: ', 'expansion rate: ', 'magnetic field decay rate: ', 'magnetic field 1 AU: ', 'drag coefficient: ', 
         'background sw speed: ']
 
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    dfAsString = out.describe().to_string(header=True, index=True)
    f.write(dfAsString)   
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for count, name in enumerate(names):
        f.write(" --{} {:.2f}".format(name, res[count+1]))
        f.write('\n')

#### loading and plotting the fitting results

In [11]:
#t_s = datetime.datetime(2023, 3, 23, 12, tzinfo=datetime.timezone.utc) 
#t_e = datetime.datetime(2023, 3, 24, 12, tzinfo=datetime.timezone.utc)

In [19]:
fp.fullinsitu(observer='Wind', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='HEEQ', best=True, 
              ensemble=True, save_fig=True, legend=True, max_index=128, title=True, fit_points=True)

INFO:py3dcore.fluxplot:Using HelioSat to retrieve observer data


2023-03-23 17:00:00+00:00 2023-03-24 15:00:00+00:00
1320


INFO:py3dcore.fluxplot:Using parameters for run with minimum eps.


[[ 0.9807951   0.00316639 -0.11960012]
 [ 0.98079769  0.00316559 -0.11959613]
 [ 0.98080028  0.0031648  -0.11959215]
 [ 0.98080287  0.003164   -0.11958816]
 [ 0.98080545  0.0031632  -0.11958417]
 [ 0.98080804  0.0031624  -0.11958017]
 [ 0.98081063  0.00316161 -0.11957618]
 [ 0.98081322  0.00316081 -0.11957218]
 [ 0.98081581  0.00316001 -0.11956818]
 [ 0.9808184   0.00315921 -0.11956418]
 [ 0.98082099  0.00315841 -0.11956018]
 [ 0.98082358  0.00315761 -0.11955618]
 [ 0.98082617  0.00315682 -0.11955217]
 [ 0.98082876  0.00315602 -0.11954816]
 [ 0.98083135  0.00315522 -0.11954416]
 [ 0.98083395  0.00315442 -0.11954014]
 [ 0.98083654  0.00315362 -0.11953613]
 [ 0.98083913  0.00315282 -0.11953212]
 [ 0.98084172  0.00315202 -0.1195281 ]
 [ 0.98084431  0.00315122 -0.11952409]
 [ 0.9808469   0.00315042 -0.11952007]
 [ 0.98084949  0.00314962 -0.11951605]
 [ 0.98085209  0.00314882 -0.11951202]
 [ 0.98085468  0.00314802 -0.119508  ]
 [ 0.98085727  0.00314721 -0.11950397]
 [ 0.98085986  0.00314641

IndexError: index 131 is out of bounds for axis 0 with size 131

In [ ]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

#### Plotting the fluxrope and spacecraft positions

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 5, 16)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist=['Earth'], t=tm1, filepath=filepath)

### make fluxrope 3d movie

In [ ]:
def make_frame(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist =['Earth'],
              t = t, filepath = filepath, save_fig = False)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/'+frmstr+'.jpg',dpi=200)  
    return fig
    

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 7, 1)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d_multiview(t_launch = t_launch, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

